## Modules

In [1]:
import os
import sys
sys.path.append('..')
from glob import glob

from time import time
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

from configparser import ConfigParser
configchain = ConfigParser()
configchain.read('setting_yahoochain.ini')
chainpath = configchain['paths'].get('chainpath')

#from bsoption.nyopchain import NYopchain
#Opchain = NYopchain()

from litedata.

## Option chain object

In [2]:
def getdayslater(date='2022-01-01', numday=0):
    """Obtain datestring format in the form '%Y-%m-%d %H:%M:%S:%f'."""
    afterdate = datetime.strptime(date, '%Y-%m-%d') + timedelta(days=numday) - timedelta(seconds=1)
    afterstr = afterdate.strftime('%Y-%m-%d %H:%M:%S:%f')
    return afterstr

class NYopchain():

    def __init__(self):
        self.engine = create_engine(f'sqlite:///{os.path.join(chainpath, "nyopchain.db")}')
        self.collist = ['tradedate', 'asset', 'optype', 'expiry', 'strike', 'iv', 'vol', 'oi', 'last', 'bid', 'ask', 'mid']
        self.colstr = '`' + '`, `'.join(self.collist) + '`'

    def getopchainyahoo(self, asset):
        """Obtain most recent trading day option chain data from yahoo finance API."""
        Ticker = yf.Ticker(asset)
        expdaylist = Ticker.options
        df1d = Ticker.history(period='1d')
        lasttd = df1d.index[0].strftime('%Y-%m-%d')

        dfchainall = pd.DataFrame()
        for daystr in expdaylist:
            expday = datetime.strptime(daystr, '%Y-%m-%d')
            chainlist = Ticker.option_chain(daystr)
            dfcall = chainlist[0]
            dfcall['optype'] = 'C'
            dfput = chainlist[1]
            dfput['optype'] = 'P'
            renamedict = {'lastPrice': 'last', 'volume': 'vol', 'openInterest': 'oi', 'impliedVolatility': 'iv'}
            for df in [dfcall, dfput]:
                df.drop(['contractSymbol','lastTradeDate','inTheMoney','contractSize','currency','change','percentChange'], 
                        axis=1, inplace=True)
                df.rename(columns=renamedict, inplace=True)
                df['asset'] = asset
                df['expiry'] = expday
            dfchain = pd.concat([dfcall, dfput], axis=0)
            dfchain['tradedate'] = lasttd
            dfchain = dfchain[self.collist[:-1]]
            for col in self.collist[4:-1]:
                dfchain[col] = pd.to_numeric(dfchain[col])
            dfchain['mid'] = (dfchain['bid'] + dfchain['ask']) / 2
            dfchain['iv'] = np.round(100 * dfchain['iv'], 2)
            dfchain.sort_values(['optype', 'strike'], inplace=True)
            dfchainall = pd.concat([dfchainall, dfchain], axis=0)

        return dfchainall

    def appendchaindf(self, dfchain):
        """Insert a dataframe of option chain."""
        assetlist = dfchain['asset'].unique()
        with self.engine.connect() as con:
            for asset in assetlist:
                dfchain1 = dfchain[dfchain['asset'] == asset]
                dfchain1.to_sql(asset, con=con, if_exists='append', index=False)

    def loadopdata(self, inputdict, orderfield=('tradedate', 'optype', 'strike',)):
        """Load option data of specific requirements."""
        stmtselect = f" SELECT {self.colstr} FROM `{inputdict['asset']}` \
                        WHERE ((`optype` = '{inputdict['optype'][0]}') OR (`optype` = '{inputdict['optype'][1]}')) \
                        AND `strike` between {inputdict['strike_lowerbound']} and {inputdict['strike_upperbound']} \
                        AND `expiry` between '{getdayslater(inputdict['startexpiry'])}' \
                        and '{getdayslater(inputdict['endexpiry'], 1)}' \
                        AND `tradedate` between '{getdayslater(inputdict['starttd'])}' \
                        and '{getdayslater(inputdict['endtd'], 1)}' \
                        ORDER BY {', '.join(orderfield)}"
        
        with self.engine.connect() as con:
            result = con.execute(stmtselect).fetchall()
        
        dfdata = pd.DataFrame(result, columns=self.collist)
        
        for col in ['tradedate', 'expiry']:
            dfdata[col] = pd.to_datetime(dfdata[col])
            
        for col in self.collist[4:]:
            dfdata[col] = pd.to_numeric(dfdata[col])

        return dfdata
    
Opchain = NYopchain()

## Integrate data into database

### Read option excel data file

In [3]:
def getchaindf(filename):
    """Obtain option chain dataframe from excel file."""
    xl = pd.ExcelFile(filename)
    tdstr = filename[-11:-5]
    tddate = datetime.strptime(tdstr, '%y%m%d')
    expirylist = xl.sheet_names
    chaindict = {}
    for expiry in expirylist:
        dfop = xl.parse(expiry)
        dfop['tradedate'] = tddate
        dfop['mid'] = (dfop['bid'] + dfop['ask']) / 2
        dfop = dfop[Opchain.collist]
        chaindict[expiry] = dfop
    return chaindict

### Inserting all other symbols

In [4]:
configassets = configchain['assets']
etflist = list(configassets.get('etf').split(','))
chipslist = list(configassets.get('chip').split(','))
assetlist = etflist + chipslist
print(assetlist)

['SPY', 'QQQ', 'IWM', 'ARKK', 'SMH', 'XLE', 'GLD', 'USO', 'TSLA', 'AAPL', 'AMZN', 'META', 'AMD', 'NVDA', 'MSFT', 'NFLX', 'SQ', 'NIO', 'BABA', 'JD', 'OXY', 'DIS', 'C', 'BA']


In [5]:
#for asset in assetlist:
#    starttime = time()
#    filelist = glob(os.path.join(chainpath, f'{asset}-*.xlsx'))
#    for filename in filelist:
#        chaindict = getchaindf(filename)
#        tdstr = filename[-11:-5]
#        for expiry, df in chaindict.items():
#            try:
#                Opchain.appendchaindf(df)
#            except:
#                print(f'Error appending {asset} option chain expiring on {expiry} on trading date {tdstr}.')
#    endtime = time()
#    totaltime = endtime - starttime
#    numfile = len(filelist)
#    avgtime = round(totaltime / numfile, 4)
#    print(f'Time elapsed inserting {numfile} files of {asset} option chains: {avgtime}s per file')

## Option chain query on one asset

### Example 1: single expiry, two adjacent trading days, only put, order by strike then tradedate

In [6]:
inputdict1 = {'asset': 'NVDA',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 150,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 180,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-08-27',  # query expiry startdate (inclusive)
              'endexpiry': '2022-09-02',  # query expiry enddate (inclusive)              
              'starttd': '2022-08-25',  # query period startdate (inclusive)
              'endtd': '2022-08-26'  # query period startdate (inclusive) 
             }

orderfield1 = ['optype', 'strike', 'tradedate']
dfop1 = Opchain.loadopdata(inputdict1, orderfield=orderfield1)
dfop1

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-08-25,NVDA,P,2022-09-02,150.0,62.89,3692.0,3178,0.17,0.15,0.18,0.165
1,2022-08-26,NVDA,P,2022-09-02,150.0,57.96,10752.0,3246,1.05,1.02,1.05,1.035
2,2022-08-25,NVDA,P,2022-09-02,152.5,61.13,685.0,1050,0.23,0.21,0.24,0.225
3,2022-08-26,NVDA,P,2022-09-02,152.5,57.30,3013.0,1337,1.48,1.47,1.49,1.480
4,2022-08-25,NVDA,P,2022-09-02,155.0,59.38,1897.0,1831,0.30,0.29,0.32,0.305
5,2022-08-26,NVDA,P,2022-09-02,155.0,56.57,5881.0,1954,2.07,2.05,2.07,2.060
6,2022-08-25,NVDA,P,2022-09-02,157.5,57.72,1360.0,1108,0.42,0.40,0.43,0.415
7,2022-08-26,NVDA,P,2022-09-02,157.5,55.71,4499.0,1299,2.77,2.77,2.81,2.790
8,2022-08-25,NVDA,P,2022-09-02,160.0,56.06,8675.0,3841,0.56,0.55,0.57,0.560
9,2022-08-26,NVDA,P,2022-09-02,160.0,54.88,26134.0,6545,3.73,3.65,3.75,3.700


### Single option contract across days

In [7]:
inputdict2 = {'asset': 'QQQ',  
              'optype': ('P', 'P'), # Option type: ("C", "C") = CALL only, ("P", "P") = PUT only, ("C", "P") = CALL & PUT
              'strike_lowerbound': 300,  # Lower bound for range of strike prices (inclusive)
              'strike_upperbound': 300,  # Upper bound for range of strike prices (inclusive)
              'startexpiry': '2022-09-16',  # query expiry startdate (inclusive)
              'endexpiry': '2022-09-16',  # query expiry enddate (inclusive)              
              'starttd': '2022-08-01',  # query period startdate (inclusive)
              'endtd': '2022-09-16'  # query period startdate (inclusive) 
             }

dfop2 = Opchain.loadopdata(inputdict2)
dfop2

,tradedate,asset,optype,expiry,strike,iv,vol,oi,last,bid,ask,mid
0,2022-08-01,QQQ,P,2022-09-16,300.0,28.61,7870.0,125557,6.29,6.24,6.28,6.260
1,2022-08-02,QQQ,P,2022-09-16,300.0,30.13,12331.0,125399,7.12,6.98,7.03,7.005
2,2022-08-03,QQQ,P,2022-09-16,300.0,30.15,10849.0,122700,4.66,4.60,4.63,4.615
3,2022-08-04,QQQ,P,2022-09-16,300.0,30.24,6000.0,125882,4.19,4.18,4.22,4.200
4,2022-08-05,QQQ,P,2022-09-16,300.0,29.58,6172.0,128425,4.44,4.42,4.47,4.445
5,2022-08-08,QQQ,P,2022-09-16,300.0,29.61,12945.0,129045,4.34,4.32,4.36,4.340
6,2022-08-09,QQQ,P,2022-09-16,300.0,29.60,12082.0,132695,5.12,5.07,5.13,5.100
7,2022-08-10,QQQ,P,2022-09-16,300.0,29.27,12717.0,128812,2.95,2.94,2.98,2.960
8,2022-08-11,QQQ,P,2022-09-16,300.0,28.99,12519.0,129764,3.12,3.10,3.13,3.115
9,2022-08-12,QQQ,P,2022-09-16,300.0,29.60,9333.0,133910,2.17,2.15,2.17,2.160
